# Lab 3: Train and hyperparameter tune on Iris Dataset with Scikit-learn
<br>

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>


Documentation : https://docs.microsoft.com/en-us/azure/machine-learning/

In this tutorial, we demonstrate how to use the Azure ML Python SDK to train a **support vector machine (SVM) on a single-node CPU with Scikit-learn to perform classification** on the popular [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris). 

We will also demonstrate how to perform **hyperparameter tuning** of the model using Azure ML's HyperDrive service.

## Prerequisites

* Go through the [Configuration](../../../../configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML Workspace

In [1]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: azuremlworkshop
Azure region: eastus
Resource group: azuremlworkshop-rg


## Create AmlCompute

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, we use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

If we could not find the cluster with the given name, then we will create a new cluster here. We will create an `AmlCompute` cluster of `STANDARD_D2_V2` CPU VMs. This process is broken down into 3 steps:
1. create the configuration (this step is local and only takes a second)
2. create the cluster (this step will take about **20 seconds**)
3. provision the VMs to bring the cluster to the initial size (of 1 in this case). This step will take about **3-5 minutes** and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster-lab3"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-11-23T17:37:33.613000+00:00', 'errors': None, 'creationTime': '2021-11-23T16:49:40.439736+00:00', 'modifiedTime': '2021-11-23T16:49:56.016039+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


The above code retrieves a CPU compute target. Scikit-learn does not support GPU computing.

## Train model on the remote compute

Now that you have your data and training script prepared, you are ready to train on your remote compute. You can take advantage of Azure compute to leverage a CPU cluster.

### Create a project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [4]:
import os
project_folder = './sklearn-iris'
os.makedirs(project_folder, exist_ok=True)

### Prepare training script

Now you will need to create your training script. In this tutorial, the training script is already provided for you at `train_iris`.py. In practice, you should be able to take any custom training script as is and run it with Azure ML without having to modify your code.

However, if you would like to use Azure ML's [tracking and metrics](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#metrics) capabilities, you will have to add a small amount of Azure ML code inside your training script.

In `train_iris.py`, we will log some metrics to our Azure ML run. To do so, we will access the Azure ML Run object within the script:

```python
from azureml.core.run import Run
run = Run.get_context()
```

Further within `train_iris.py`, we log the kernel and penalty parameters, and the highest accuracy the model achieves:

```python
run.log('Kernel type', np.string(args.kernel))
run.log('Penalty', np.float(args.penalty))

run.log('Accuracy', np.float(accuracy))
```

These run metrics will become particularly important when we begin hyperparameter tuning our model in the "Tune model hyperparameters" section.

Once your script is ready, copy the training script `train_iris.py` into your project directory.

In [5]:
import shutil
shutil.copy('train_iris.py', project_folder)

'./sklearn-iris/train_iris.py'

### Create an experiment

Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this Scikit-learn tutorial.

In [6]:
from azureml.core import Experiment
experiment_name = 'Lab3-ScikitLearn'
experiment = Experiment(ws, name=experiment_name)

### Create an environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [7]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Overwriting conda_dependencies.yml


In [8]:
!ls conda_dependencies.yml -l

-rwxrwxrwx 1 root root 73 Dec  1 09:07 conda_dependencies.yml


In [9]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

### Configure the training job

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

In [10]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='train_iris.py',
                      arguments=['--kernel', 'linear', '--penalty', 1.0],
                      compute_target=compute_target,
                      environment=sklearn_env)

### Submit job

Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [11]:
run = experiment.submit(src)

## Monitor your run

You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [12]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [13]:
run.wait_for_completion(show_output=True)

RunId: Lab3-ScikitLearn_1638349691_2ce1e628
Web View: https://ml.azure.com/runs/Lab3-ScikitLearn_1638349691_2ce1e628?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azuremlworkshop-rg/workspaces/azuremlworkshop&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Execution Summary
RunId: Lab3-ScikitLearn_1638349691_2ce1e628
Web View: https://ml.azure.com/runs/Lab3-ScikitLearn_1638349691_2ce1e628?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azuremlworkshop-rg/workspaces/azuremlworkshop&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Warnings:
This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"




{'runId': 'Lab3-ScikitLearn_1638349691_2ce1e628',
 'target': 'cpu-cluster-lab3',
 'status': 'Completed',
 'startTimeUtc': '2021-12-01T09:10:24.618819Z',
 'endTimeUtc': '2021-12-01T09:13:38.133483Z',
 'warnings': [{'message': 'This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}],
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '56d28dd8-9134-436a-889c-1547fd7fec91',
  'azureml.git.repository_uri': 'https://github.com/retkowsky/AMLlabs',
  'mlflow.source.git.repoURL': 'https://github.com/retkowsky/AMLlabs',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'fe66fa1ed3e8aae185ea1905

## Tune model hyperparameters

Now that we've seen how to do a simple Scikit-learn training run using the SDK, let's see if we can further improve the accuracy of our model. We can optimize our model's hyperparameters using Azure Machine Learning's hyperparameter tuning capabilities.

### Start a hyperparameter sweep

First, we will define the hyperparameter space to sweep over. Let's tune the `kernel` and `penalty` parameters. In this example we will use random sampling to try different configuration sets of hyperparameters to maximize our primary metric, `Accuracy`.

In [14]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
    
param_sampling = RandomParameterSampling( {
    "--kernel": choice('linear', 'rbf', 'poly', 'sigmoid'),
    "--penalty": choice(0.5, 1, 1.5)
    }
)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 12,
                                     max_concurrent_runs = 4)

Finally, lauch the hyperparameter tuning job.

In [15]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

## Monitor HyperDrive runs

You can monitor the progress of the runs with the following Jupyter widget.

In [16]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_bda721f2-5a99-46ef-be07-90d97b5224ac
Web View: https://ml.azure.com/runs/HD_bda721f2-5a99-46ef-be07-90d97b5224ac?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azuremlworkshop-rg/workspaces/azuremlworkshop&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-12-01T10:01:42.184769][API][INFO]Experiment created<END>\n""<START>[2021-12-01T10:01:42.955659][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-12-01T10:01:43.105895][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_bda721f2-5a99-46ef-be07-90d97b5224ac
Web View: https://ml.azure.com/runs/HD_bda721f2-5a99-46ef-be07-90d97b5224ac?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azuremlworkshop-rg/workspaces/azuremlworkshop&tid=72f988bf-86f1-41af-91ab-2d7cd011db47



{'runId': 'HD_bda721f2-5a99-46ef-be07-90d97b5224ac',
 'target': 'cpu-cluster-lab3',
 'status': 'Completed',
 'startTimeUtc': '2021-12-01T10:01:42.035106Z',
 'endTimeUtc': '2021-12-01T10:13:26.461446Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '0833db63-f971-4916-a4b1-8d43bd7854b1',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.9736842105263158',
  'best_child_run_id': 'HD_bda721f2-5a99-46ef-be07-90d97b5224ac_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://azuremlworksho7897742112.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_bda721f2-5a99-46ef-be07-90d97b5224ac/azureml-logs/hyperdrive.txt?sv=

In [18]:
assert(hyperdrive_run.get_status() == "Completed")

### Warm start a Hyperparameter Tuning experiment and resuming child runs
Often times, finding the best hyperparameter values for your model can be an iterative process, needing multiple tuning runs that learn from previous hyperparameter tuning runs. Reusing knowledge from these previous runs will accelerate the hyperparameter tuning process, thereby reducing the cost of tuning the model and will potentially improve the primary metric of the resulting model. When warm starting a hyperparameter tuning experiment with Bayesian sampling, trials from the previous run will be used as prior knowledge to intelligently pick new samples, so as to improve the primary metric. Additionally, when using Random or Grid sampling, any early termination decisions will leverage metrics from the previous runs to determine poorly performing training runs. 

Azure Machine Learning allows you to warm start your hyperparameter tuning run by leveraging knowledge from up to 5 previously completed hyperparameter tuning parent runs. 

Additionally, there might be occasions when individual training runs of a hyperparameter tuning experiment are cancelled due to budget constraints or fail due to other reasons. It is now possible to resume such individual training runs from the last checkpoint (assuming your training script handles checkpoints). Resuming an individual training run will use the same hyperparameter configuration and mount the storage used for that run. The training script should accept the "--resume-from" argument, which contains the checkpoint or model files from which to resume the training run. You can also resume individual runs as part of an experiment that spends additional budget on hyperparameter tuning. Any additional budget, after resuming the specified training runs is used for exploring additional configurations.

For more information on warm starting and resuming hyperparameter tuning runs, please refer to the [Hyperparameter Tuning for Azure Machine Learning documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters) 

### Find and register best model
When all jobs finish, we can find out the one that has the highest accuracy.

In [19]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--kernel', 'linear', '--penalty', '1', '--kernel', 'linear', '--penalty', '1']


Now, let's list the model files uploaded during the run.

In [20]:
print(best_run.get_file_names())

['logs/azureml/19_azureml.log', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


We can then register the folder (and all files in it) as a model named `sklearn-iris` under the workspace for deployment

In [21]:
from azureml.core import Model

best_run.register_model(model_path='outputs/model.joblib', 
                        model_name='sklearn-iris')

Model(workspace=Workspace.create(name='azuremlworkshop', subscription_id='70b8f39e-8863-49f7-b6ba-34a80799550c', resource_group='azuremlworkshop-rg'), name=sklearn-iris, id=sklearn-iris:3, version=3, tags={}, properties={})

> You can open now the **Lab4 notebook**.

<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">